# Part 1: Feature Backfill for Air Quality and Weather Data

In [20]:
import sys
from pathlib import Path
import warnings

warnings.filterwarnings("ignore", module="IPython")

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    if root_dir.parts[-1:] == ("src",):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ("airquality",):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ("notebooks",):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir)
    print("Local environment")

print(f"Root dir: {root_dir}")

if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

from utils import config

settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Root dir: /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn
HopsworksSettings initialized!


## Imports

In [21]:
import datetime
import time
import requests
import pandas as pd
import great_expectations as ge
import hopsworks
from utils import airquality
import datetime
import json
import os
import warnings
warnings.filterwarnings("ignore")

## Setup

In [22]:
today = datetime.date.today()
project = hopsworks.login(engine="python")
fs = project.get_feature_store()

# taken from ~/.env You can also replace settings.AQICN_API_KEY with the api key value as a string "..."
if settings.AQICN_API_KEY is None:
    print("You need to set AQICN_API_KEY either in this cell or in ~/.env")
    sys.exit(1)

AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value()

print(f"Found AQICN_API_KEY: {AQICN_API_KEY}")

secrets = hopsworks.get_secrets_api()
try:
    secret = secrets.get_secret("AQICN_API_KEY")
    if secret is not None:
        secret.delete()
        print("Replacing existing AQICN_API_KEY")
except hopsworks.RestAPIError as e:
    if hasattr(e, "error_code") and getattr(e, "error_code", None) == 160048:
        pass
    elif "Could not find Secret" in str(e):
        pass
    else:
        raise

secrets.create_secret("AQICN_API_KEY", AQICN_API_KEY)

2025-11-17 14:36:07,780 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-17 14:36:07,785 INFO: Initializing external client
2025-11-17 14:36:07,785 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-17 14:36:09,281 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279179
Found AQICN_API_KEY: 17a266deb78b8d8e7b8ad87076443a4e7ee32801
Replacing existing AQICN_API_KEY
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('AQICN_API_KEY', 'PRIVATE')

Set SENSOR_CSV_FILE in .env with the relative path to a sensor to process it, or leave it unset to process all sensors in the `data` folder

In [23]:
sensor_csv_file = getattr(settings, 'SENSOR_CSV_FILE', None)

In [24]:
aq_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)

aq_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "pm25",
            "min_value": -0.1,
            "max_value": 500.0,
            "strict_min": True,
        },
    )
)

weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

def expect_greater_than_zero(col):
    weather_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column": col,
                "min_value": -0.1,
                "max_value": 1000.0,
                "strict_min": True,
            },
        )
    )


expect_greater_than_zero("precipitation_sum")
expect_greater_than_zero("wind_speed_10m_max")

## Helper Methods

In [25]:
def clean_and_append_data(df, street, city, country, feed_url, sensor_id):
    """
    Remove any unused columns, set the daily median value to pm25. Remove NaN's and append the metadata.
    """
    clean_df = pd.DataFrame()
    clean_df["date"] = pd.to_datetime(df["date"])
    clean_df["pm25"] = df["median"]
    clean_df = clean_df.dropna(subset=["pm25"])
    clean_df["sensor_id"] = sensor_id
    clean_df["street"] = street
    clean_df["city"] = city
    clean_df["country"] = country
    clean_df["feed_url"] = feed_url
    return clean_df


def get_historical_weather(city, df, today, feed_url, sensor_id):
    earliest_aq_date = pd.Series.min(df["date"])
    earliest_aq_date = earliest_aq_date.strftime("%Y-%m-%d")
    response = requests.get(f"{feed_url}/?token={AQICN_API_KEY}")
    data = response.json()
    latitude = data["data"]["city"]["geo"][0]
    longitude = data["data"]["city"]["geo"][1]
    max_retries = 5
    attempt = 0
    while attempt < max_retries:
        try:
            weather_df = airquality.get_historical_weather(
                city, earliest_aq_date, str(today), latitude, longitude
            )
            weather_df["sensor_id"] = sensor_id
            weather_df["city"] = city
            weather_df["latitude"] = latitude
            weather_df["longitude"] = longitude
            return weather_df, latitude, longitude
        except Exception as e:
            if hasattr(e, "args") and any(
                "Minutely API request limit exceeded" in str(a) for a in e.args
            ):
                wait_time = 70
                print(
                    f"OpenMeteo API limit exceeded, retrying in {wait_time} seconds... (Attempt {attempt + 1} of {max_retries})"
                )
                time.sleep(wait_time)
                attempt += 1
            elif "Minutely API request limit exceeded" in str(e):
                wait_time = 70
                print(
                    f"OpenMeteo API limit exceeded, retrying in {wait_time} seconds... (Attempt {attempt + 1} of {max_retries})"
                )
                time.sleep(wait_time)
                attempt += 1
            else:
                raise
    raise RuntimeError(
        "Failed to obtain historical weather after multiple retries due to API rate limits."
    )

## Hopsworks

In [26]:
def create_air_quality_feature_group():
    air_quality_fg = fs.get_or_create_feature_group(
        name="air_quality_all",
        description="Air Quality characteristics of each day for all sensors",
        version=1,
        primary_key=["sensor_id"],
        event_time="date",
        expectation_suite=aq_expectation_suite,
    )
    return air_quality_fg


def update_air_quality_description(air_quality_fg):
    air_quality_fg.update_feature_description(
        "date", "Date of measurement of air quality"
    )
    air_quality_fg.update_feature_description(
        "sensor_id", "AQICN sensor identifier (e.g., 59893)"
    )
    air_quality_fg.update_feature_description(
        "country",
        "Country where the air quality was measured (sometimes a city in aqicn.org)",
    )
    air_quality_fg.update_feature_description(
        "city", "City where the air quality was measured"
    )
    air_quality_fg.update_feature_description(
        "street", "Street in the city where the air quality was measured"
    )
    air_quality_fg.update_feature_description(
        "pm25",
        "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk",
    )
    air_quality_fg.update_feature_description(
        "pm25_rolling_3d",
        "3-day rolling mean of PM2.5 from previous days (lagged by 1 day for point-in-time correctness).",
    )
    air_quality_fg.update_feature_description(
        "pm25_lag_1d",
        "PM2.5 value from 1 day ago.",
    )
    air_quality_fg.update_feature_description(
        "pm25_lag_2d",
        "PM2.5 value from 2 days ago.",
    )
    air_quality_fg.update_feature_description(
        "pm25_lag_3d",
        "PM2.5 value from 3 days ago.",
    )
    air_quality_fg.update_feature_description(
        "pm25_nearby_avg",
        "Average PM2.5 value from the 3 closest sensors.",
    )


def create_and_insert_air_quality_data(df):
    air_quality_fg = create_air_quality_feature_group()
    air_quality_fg.insert(df)
    update_air_quality_description(air_quality_fg)


def create_weather_feature_group():
    weather_fg = fs.get_or_create_feature_group(
        name="weather_all",
        description="Weather characteristics of each day for all sensors",
        version=1,
        primary_key=["sensor_id"],
        event_time="date",
        expectation_suite=weather_expectation_suite,
    )
    return weather_fg


def update_weather_description(weather_fg):
    weather_fg.update_feature_description("date", "Date of measurement of weather")
    weather_fg.update_feature_description(
        "sensor_id", "AQICN sensor identifier (e.g., 59893)"
    )
    weather_fg.update_feature_description(
        "city", "City where weather is measured/forecast for"
    )
    weather_fg.update_feature_description(
        "temperature_2m_mean", "Temperature in Celsius"
    )
    weather_fg.update_feature_description(
        "precipitation_sum", "Precipitation (rain/snow) in mm"
    )
    weather_fg.update_feature_description(
        "wind_speed_10m_max", "Wind speed at 10m abouve ground"
    )
    weather_fg.update_feature_description(
        "wind_direction_10m_dominant", "Dominant Wind direction over the dayd"
    )
    weather_fg.update_feature_description(
        "latitude", "Latitude of sensor location used for weather retrieval"
    )
    weather_fg.update_feature_description(
        "longitude", "Longitude of sensor location used for weather retrieval"
    )


def create_and_insert_weather_data(df):
    weather_fg = create_weather_feature_group()
    weather_fg.insert(df)
    update_weather_description(weather_fg)

## Script

In [27]:
all_aq_dfs = []
all_weather_dfs = []
locations = {}

if sensor_csv_file and os.path.exists(sensor_csv_file):
    # Single sensor mode if csv_file provided
    aq_df_raw, street, city, country, feed_url, sensor_id = airquality.read_sensor_data(sensor_csv_file)
    aq_df = clean_and_append_data(aq_df_raw, street, city, country, feed_url, sensor_id)
    weather_df, latitude, longitude = get_historical_weather(
        city, aq_df, today, feed_url, sensor_id
    )
    all_aq_dfs.append(aq_df)
    all_weather_dfs.append(weather_df)
    locations[sensor_id] = {
        "country": country,
        "city": city,
        "street": street,
        "aqicn_url": feed_url,
        "latitude": latitude,
        "longitude": longitude,
    }
else:
    # Process all files in data directory
    data_dir = os.path.join(root_dir, "data")
    dir_list = os.listdir(data_dir)
    for file in dir_list:
        file_path = os.path.join(data_dir, file)
        aq_df_raw, street, city, country, feed_url, sensor_id = airquality.read_sensor_data(file_path)
        aq_df = clean_and_append_data(aq_df_raw, street, city, country, feed_url, sensor_id)
        weather_df, latitude, longitude = get_historical_weather(
            city, aq_df, today, feed_url, sensor_id
        )
        all_aq_dfs.append(aq_df)
        all_weather_dfs.append(weather_df)
        locations[sensor_id] = {
            "country": country,
            "city": city,
            "street": street,
            "aqicn_url": feed_url,
            "latitude": latitude,
            "longitude": longitude,
        }

# Concatenate into single, uniform dfs
aq_df_all = pd.concat(all_aq_dfs, ignore_index=True)
weather_df_all = pd.concat(all_weather_dfs, ignore_index=True)
aq_df_all = airquality.add_rolling_window_feature(aq_df_all, window_days=3, column="pm25", new_column="pm25_rolling_3d")
aq_df_all = airquality.add_lagged_features(aq_df_all, column="pm25", lags=[1, 2, 3])
aq_df_all = airquality.add_nearby_sensor_feature(aq_df_all, locations, column="pm25_lag_1d", n_closest=3)

In [28]:
aq_df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28290 entries, 14572 to 4062
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   date             28290 non-null  datetime64[ns, UTC]
 1   pm25             28290 non-null  float64            
 2   sensor_id        28290 non-null  object             
 3   street           28290 non-null  object             
 4   city             28290 non-null  object             
 5   country          28290 non-null  object             
 6   feed_url         28290 non-null  object             
 7   pm25_rolling_3d  28275 non-null  float64            
 8   pm25_lag_1d      28275 non-null  float64            
 9   pm25_lag_2d      28260 non-null  float64            
 10  pm25_lag_3d      28245 non-null  float64            
 11  pm25_nearby_avg  28272 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(6), object(5)
memory usage: 2.8+ MB


In [29]:
weather_df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30421 entries, 0 to 30420
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         30421 non-null  datetime64[ns]
 1   temperature_2m_mean          30421 non-null  float32       
 2   precipitation_sum            30421 non-null  float32       
 3   wind_speed_10m_max           30421 non-null  float32       
 4   wind_direction_10m_dominant  30421 non-null  float32       
 5   city                         30421 non-null  object        
 6   sensor_id                    30421 non-null  object        
 7   latitude                     30421 non-null  float64       
 8   longitude                    30421 non-null  float64       
dtypes: datetime64[ns](1), float32(4), float64(2), object(2)
memory usage: 1.6+ MB


In [30]:
# Create individual secrets for each sensor
for sensor_id, location in locations.items():
    secret_name = f"SENSOR_LOCATION_JSON_{sensor_id}"
    location_str = json.dumps(location)
    
    try:
        secret = secrets.get_secret(secret_name)
        if secret is not None:
            secret.delete()
            print(f"Replacing existing {secret_name}")
    except hopsworks.RestAPIError as e:
        if hasattr(e, "error_code") and getattr(e, "error_code", None) == 160048:
            pass
        elif "Could not find Secret" in str(e):
            pass
        else:
            raise
    
    secrets.create_secret(secret_name, location_str)
    print(f"Created secret: {secret_name}\n")

Replacing existing SENSOR_LOCATION_JSON_59893
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets
Created secret: SENSOR_LOCATION_JSON_59893

Replacing existing SENSOR_LOCATION_JSON_88372
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets
Created secret: SENSOR_LOCATION_JSON_88372

Replacing existing SENSOR_LOCATION_JSON_69628
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets
Created secret: SENSOR_LOCATION_JSON_69628

Replacing existing SENSOR_LOCATION_JSON_61714
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets
Created secret: SENSOR_LOCATION_JSON_61714

Replacing existing SENSOR_LOCATION_JSON_60541
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets
Created secret: SENSOR_LOCATION_JSON_60541

Replacing existing SENSOR_LOCATION_JSON_404209
Secret created successfully, explore it at https://c.

In [31]:
# Insert unified dfs into single feature groups
create_and_insert_air_quality_data(aq_df_all)
create_and_insert_weather_data(weather_df_all)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1279179/fs/1265797/fg/1721857
2025-11-17 14:36:47,715 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279179/fs/1265797/fg/1721857


Uploading Dataframe: 100.00% |██████████| Rows 28290/28290 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: air_quality_all_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279179/jobs/named/air_quality_all_1_offline_fg_materialization/executions
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1279179/fs/1265797/fg/1718803
2025-11-17 14:37:21,711 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279179/fs/1265797/fg/1718803


Uploading Dataframe: 100.00% |██████████| Rows 30421/30421 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: weather_all_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279179/jobs/named/weather_all_1_offline_fg_materialization/executions
